In [1]:
!pip -q install langchain langchain-community
!pip -q install pypdf
!pip -q install sentence_transformers
!pip install openai
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.1 MB/s eta 0:00:00


In [2]:
!pip install tokenizers
!pip install faiss-cpu
!pip -q install unstructured

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 8.9 MB/s eta 0:00:00


In [3]:
!pip install numpy==1.24.4
!pip install nltk==3.9.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 86.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
faiss-cpu 1.9.0.post1 requires numpy<3.0,>=1.25.0, but you have numpy 1.24.4 which is incompatible.
pymc 5.19.1 requires numpy>=1.25.0, but you have numpy 1.24.4 which is incompatible.


In [1]:
import sys
import os
import torch
import textwrap
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.embeddings import HuggingFaceEmbeddings

In [2]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
URLs=[
    'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb',
    'https://www.mosaicml.com/blog/mpt-7b',
    'https://stability.ai/blog/stability-ai-launches-the-first-of-its-stablelm-suite-of-language-models',
    'https://lmsys.org/blog/2023-03-30-vicuna/'

]

In [5]:
loaders = UnstructuredURLLoader(urls=URLs)
data = loaders.load()

In [7]:
len(data)

4

In [8]:
text_splitter=CharacterTextSplitter(separator='\n',
                                    chunk_size=1000,
                                    chunk_overlap=200)

In [10]:
text_chunks=text_splitter.split_documents(data)

In [12]:
text_chunks[0]

Document(metadata={'source': 'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb'}, page_content='Open in app\nSign up\nSign in\nWrite\nSign up\nSign in\nPaper Review\nPaper Review: Llama 2: Open Foundation and Fine-Tuned Chat Models\nLlama 2: one of the best open source models\nAndrew Lukyanenko\nFollow\nPublished in\nGoPenAI\n15 min read\nJul 20, 2023\n--\nProject link\nModel link\nPaper link\nThe authors of the work present Llama 2, an assortment of pretrained and fine-tuned large language models (LLMs) with sizes varying from 7 billion to 70 billion parameters. The fine-tuned versions, named Llama 2-Chat, are specifically designed for dialogue applications. These models surpass the performance of existing open-source chat models on most benchmarks, and according to human evaluations for usefulness and safety, they could potentially replace closed-source models. The authors also detail their approach to fine-tuning and safety enhanc

In [13]:
embeddings = OpenAIEmbeddings()

<ipython-input-13-73ad2f8e367a>:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [15]:
query_result = embeddings.embed_query("Hello world")
len(query_result)

1536

In [16]:
vectorstore=FAISS.from_documents(text_chunks, embeddings)

In [17]:
llm=ChatOpenAI()

<ipython-input-17-bd12acb39de2>:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm=ChatOpenAI()


In [19]:
llm.predict("Please provide a concise summary of the Book Harry Potter")

<ipython-input-19-181e585c8cae>:1: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm.predict("Please provide a concise summary of the Book Harry Potter")


'Harry Potter is a series of seven fantasy novels written by British author J.K. Rowling. The story follows the life of a young wizard, Harry Potter, and his friends Hermione Granger and Ron Weasley as they attend Hogwarts School of Witchcraft and Wizardry and battle against the dark wizard Lord Voldemort. The books explore themes of friendship, love, courage, and the power of good versus evil. Throughout the series, Harry learns about his own identity and destiny as he fights against the forces of darkness.'

In [20]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorstore.as_retriever())

In [21]:
result=chain({"question": "How good is Vicuna?"}, return_only_outputs=True)

<ipython-input-21-6dc5d9fb268a>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result=chain({"question": "How good is Vicuna?"}, return_only_outputs=True)


In [25]:
result['answer']

'Vicuna is considered to have competitive performance compared to other open-source models like Stanford Alpaca and is rated highly in quality compared to ChatGPT.\n'

In [26]:
wrapped_text = textwrap.fill(result['answer'], width=500)

In [27]:
wrapped_text

'Vicuna is considered to have competitive performance compared to other open-source models like Stanford Alpaca and is rated highly in quality compared to ChatGPT.'

In [28]:
result=chain({"question": "How does Llama 2 outperforms other models"}, return_only_outputs=True)

In [29]:
result['answer']

'Llama 2 outperforms other models in various benchmarks and surpasses the performance of existing open-source chat models, potentially even replacing closed-source models.\n'

In [30]:
result=chain({"question": "What is is stableLM?"}, return_only_outputs=True)


In [31]:
result['answer']

'StableLM is an open-source language model developed by Stability AI and trained on a new experimental dataset built on The Pile, three times larger with 1.5 trillion content tokens. It is available in various parameter sizes and can be used for commercial or research purposes. Additionally, MosaicML has also developed a model called StableLM that has been trained on 1 trillion tokens with specific features.\n'

In [ ]:
while True:
  query=input(f"Prompt: ")
  if query == 'exit':
    print('Exiting')
    sys.exit()
  if query =='':
    continue
  result=chain({'question':query})
  print(f"Answer: " +result["answer"])

Prompt: hello
Answer: I don't know.

Prompt: whats vicuna?
Answer: Vicuna is an open-source chatbot that has been fine-tuned with user-shared conversations to generate detailed and well-structured answers compared to Alpaca. It is part of the Vicuna-13B model, which is backed by an enhanced dataset and scalable infrastructure. Vicuna has limitations in tasks involving reasoning or mathematics and has not been optimized for safety or mitigating potential toxicity or bias.

